In [1]:
import datetime
datetime.datetime.now().isoformat()

'2020-05-26T21:11:25.332383'

In [2]:
import pandas as pd
import requests
import json

### Get latest data

In [3]:
url = "https://developer-apis.awair.is/v1/users/self/devices/awair-element/1714/air-data/latest?fahrenheit=TRUE"
access_token = file1 = open('/Users/matthewkudija/Downloads/awair-access-token.txt','r').read()
payload = {}
headers = {
  'Authorization': 'Bearer '+access_token
}

response = requests.request("GET", url, headers=headers, data = payload)

print(response.text.encode('utf8'))

b'{"data":[{"timestamp":"2020-05-27T01:11:24.336Z","score":89.0,"sensors":[{"comp":"temp","value":78.96200027465821},{"comp":"humid","value":52.08000183105469},{"comp":"co2","value":602.0},{"comp":"voc","value":32.0},{"comp":"pm25","value":3.0}],"indices":[{"comp":"temp","value":1.0},{"comp":"humid","value":1.0},{"comp":"co2","value":1.0},{"comp":"voc","value":0.0},{"comp":"pm25","value":0.0}]}]}'


### 5-Min Avg Air Data

In [4]:
url = "https://developer-apis.awair.is/v1/users/self/devices/awair-element/1714/air-data/5-min-avg?from=2020-05-26T10:35:00&to=2020-05-26T10:40:00&limit=288&desc=TRUE&fahrenheit=TRUE"

payload = {}
headers = {
  'Authorization': 'Bearer '+access_token
}

response = requests.request("GET", url, headers=headers, data = payload)

print(response.text.encode('utf8'))

b'{"data":[{"timestamp":"2020-05-26T10:35:00.000Z","score":85.30000305175781,"sensors":[{"comp":"temp","value":75.4879997253418},{"comp":"humid","value":59.77399826049805},{"comp":"co2","value":908.2000122070312},{"comp":"voc","value":205.76666259765625},{"comp":"pm25","value":3.8333332538604736}],"indices":[{"comp":"temp","value":0.0},{"comp":"humid","value":1.0},{"comp":"co2","value":1.0},{"comp":"voc","value":0.0},{"comp":"pm25","value":0.0}]}]}'


### 15-Min Avg Air Data

In [5]:
url = "https://developer-apis.awair.is/v1/users/self/devices/awair-element/1714/air-data/15-min-avg?from=2020-05-26T10:30:00&to=2020-05-26T10:45:00&limit=672&desc=TRUE&fahrenheit=TRUE"

payload = {}
headers = {
  'Authorization': 'Bearer '+access_token
}

response = requests.request("GET", url, headers=headers, data = payload)

print(response.text.encode('utf8'))

b'{"data":[{"timestamp":"2020-05-26T10:30:00.000Z","score":84.71302795410156,"sensors":[{"comp":"temp","value":75.58893089294433},{"comp":"humid","value":60.103424072265625},{"comp":"co2","value":919.7360229492188},{"comp":"voc","value":217.5808359781901},{"comp":"pm25","value":3.7448275883992515}],"indices":[{"comp":"temp","value":0.0},{"comp":"humid","value":1.2643678188323975},{"comp":"co2","value":1.0},{"comp":"voc","value":0.057471265395482383},{"comp":"pm25","value":0.0}]}]}'


In [6]:
data = json.loads(response.text)['data']
timestamp = data[0]['timestamp']
score = round(data[0]['score'],1)

In [7]:
# sensor data
from pandas.io.json import json_normalize 

sensors = (json_normalize(data, 'sensors')
      .set_index('comp')
      .T
      .round(1)
      .reset_index(drop=True)
     )
print(sensors)

comp  temp  humid    co2    voc  pm25
0     75.6   60.1  919.7  217.6   3.7


In [8]:
# detailed scores
from pandas.io.json import json_normalize 

scores = (json_normalize(data, 'indices')
      .set_index('comp')
      .T
      .round(1)
      .reset_index(drop=True)
      .rename(columns={'temp': 'temp_score',
                       'humid': 'humid_score',
                       'co2': 'co2_score',
                       'voc': 'voc_score',
                       'pm25': 'pm25'
                      })
     )

print(scores)

comp  temp_score  humid_score  co2_score  voc_score  pm25
0            0.0          1.3        1.0        0.1   0.0


In [9]:
df = (pd.DataFrame([[timestamp, score]], 
                  columns = ['timestamp', 'score'])
#        .set_index('timestamp')
     )
df

,timestamp,score
0,2020-05-26T10:30:00.000Z,84.7


In [10]:
df = pd.concat([df,sensors,scores],
               axis=1,
               sort=True
              )
df

,timestamp,score,temp,humid,co2,voc,pm25,temp_score,humid_score,co2_score,voc_score,pm25
0,2020-05-26T10:30:00.000Z,84.7,75.6,60.1,919.7,217.6,3.7,0.0,1.3,1.0,0.1,0.0
